In [1]:
words = []

with open('words.txt') as f:
    words = f.readlines()

In [3]:
with open("words.csv", "w") as out:
    out.write("1st,2nd,3rd,4th,5th\n")
    
    for word in words:
        test = list(word)
        if "\n" in test:
            test.remove("\n")
        out.write(",".join(test))
        out.write("\n")

In [5]:
import re
import sys
import pandas as pd
import random

startsWithDigit = re.compile("^\d")
containsNonAlpha = re.compile("\W")

def GeNIefy (term,node = None):
    if startsWithDigit.match(term) and node is not None:
        tmp = node + "_" + term
    else: 
        tmp = term
    if containsNonAlpha.search(tmp):
        tmp = containsNonAlpha.sub("_",tmp)
    return tmp

filein = "words"
fileout = "words"

# +
df = pd.read_csv(filein + ".csv",sep=',')
df = df.dropna(how='all').fillna('')

with open(fileout + ".xdsl",'w') as fo:
    fo.write("""<?xml version="1.0" encoding="UTF-8"?>
<!-- This network was created in GeNIe Academic, which can be used for academic teaching and research purposes only -->
<smile version="1.0" id="Network1" numsamples="10000" discsamples="10000">
    <nodes>\n""")
    extensions = """\t<extensions>\n\t\t<genie version="1.0" app="GeNIe 3.0.6128.0 ACADEMIC" name="Network1">"""
    for column in df:
        node = column
        if column == "fnlwgt":
            continue
        atts = list(df[column].unique())
        GeNIeNode = GeNIefy(node)
        #if "continuous" in atts:
            #just for skipping continous values vars which need to be modelled than by hand
            #continue
        fo.write('<cpt id="' + GeNIefy(node) + '">' + "\n")
        empty = 0
        for (i,att) in enumerate(atts):
            if att == "":
                empty = 1
                continue
            fo.write('  <state id="' + GeNIefy(att,GeNIeNode) + '" />' + "\n")
        fo.write("\t<probabilities>") 
        for (i,att) in enumerate(atts):
            if i == len(atts)-empty:
                continue
            fo.write("{:1.9f} ".format(1/(len(atts)-empty)))
        fo.write("</probabilities>\n") 
        fo.write("</cpt>\n")
        # Construct Extensions
        posxmin = i*int(50*random.random())+int(30*random.random())
        posxmax = posxmin+100
        posymin = i*int(30*random.random())+int(30*random.random())
        posymax = posymin+30
        extensions += """<node id="{}">
            <name>{}</name>
            <interior color="e5f6f7" />
            <outline color="000080" />
            <font color="000000" name="Arial" size="8" />
            <position>{:d} {:d} {:d} {:d}</position>
        </node>\n""".format(GeNIeNode,node,posxmin,posymin,posxmax,posymax)
    fo.write("</nodes>\n")
    fo.write(extensions)
    fo.write("</genie> </extensions> \n")
    fo.write("</smile>\n")

